In [1]:
%load_ext cuml.accel

cuML: Accelerator installed.


In [2]:
import rmm
rmm.reinitialize(
    pool_allocator=True,
    initial_pool_size=20<<30,   # 20 GiB pool (tweak as you like)
    managed_memory=False
)
rmm.mr.set_current_device_resource(rmm.mr.ManagedMemoryResource())

In [3]:
import numpy as np
import pandas as pd
import sklearn as sk
import seaborn as sns
import matplotlib.pyplot as plt
import unicodedata
import warnings
import pickle
warnings.simplefilter(action='ignore', category=FutureWarning)

In [4]:
Songs = []
with open("Songs.pkl", 'rb') as f:
    Songs = pickle.load(f) 
data = [vars(song) for song in Songs]
df = pd.DataFrame(data)
df['mfccs_path'] = df['mfccs_path'].str.replace("/Users/tylerho/Library/CloudStorage/GoogleDrive-tylerho@stanford.edu/.shortcut-targets-by-id/11Wd8pqP4BVeS--hw1VHHo4r5uRk9L1JP/K-pop Project 2024-5/K-pop Project/song_mfccs", "/home/tyler/Downloads/kpop_project/song_mfccs", regex=False)
df['mfccs_path'] = df['mfccs_path'].apply(lambda mfccs_path: unicodedata.normalize('NFC', mfccs_path))
df['mfccs_path'][0]

'/home/tyler/Downloads/kpop_project/song_mfccs/TRAX - Are you Ready¿.mp3.npy'

In [6]:
# First, we must clean the data (specifically make the mfccs uniform length)
mfccs = [np.load(mfccs_path) for mfccs_path in df['mfccs_path']]
lengths = [mfcc.shape[1] for mfcc in mfccs]
max_len = max(lengths)
percentile_95 = int(np.percentile(lengths, 95))
print(max_len, percentile_95)

53604 12316


In [7]:
def load_pad_trunc(path, T_max):
    """
    Load an MFCC matrix of shape (n_mfcc, T) from `path`,
    truncate to T_max if T > T_max, else zero-pad on the right.
    Returns a 1D array of length n_mfcc * T_max.
    """
    if path != '':
        mfcc = np.load(path)             # shape: (n_mfcc, T)
        n_mfcc, T = mfcc.shape
        if T >= T_max:
            mfcc2 = mfcc[:, :T_max]
        else:
            pad_width = T_max - T
            mfcc2 = np.pad(mfcc,
                        pad_width=((0, 0), (0, pad_width)),
                        mode='constant',
                        constant_values=0)
        return mfcc2.ravel()             # shape: (n_mfcc*T_max,)
    else:
        return np.array([1])

In [8]:
# make all the mfccs the same length
fixed_length = percentile_95
# fixed_length = 5000
feature_list = df['mfccs_path'].apply(lambda p: load_pad_trunc(p, fixed_length))


In [9]:
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from cuml.neighbors import KNeighborsClassifier
from cuml.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix

X = np.vstack(feature_list)
y = df['generation'].values.astype("int32")

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42)

scaler = StandardScaler().fit(X_train)
X_train_s, X_test_s = scaler.transform(X_train), scaler.transform(X_test)

knn = KNeighborsClassifier(n_neighbors=3, weights='uniform', metric='euclidean')
knn.fit(X_train_s, y_train)

y_pred = knn.predict(X_test_s)
print(classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           1       0.48      0.34      0.40       252
           2       0.48      0.45      0.46       362
           3       0.40      0.59      0.48       335
           4       0.53      0.28      0.36       144
           5       0.09      0.15      0.11        13

    accuracy                           0.44      1106
   macro avg       0.39      0.36      0.36      1106
weighted avg       0.46      0.44      0.43      1106

Confusion Matrix:
 [[ 85  65  93   7   2]
 [ 39 162 143  10   8]
 [ 37  77 197  18   6]
 [ 12  33  54  40   5]
 [  5   2   3   1   2]]


In [10]:
# import cupy
# from rmm.allocators.cupy_allocator import rmm_cupy_allocator
# cupy.cuda.set_allocator(rmm_cupy_allocator)

In [12]:
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_val_score(knn, X_train, y_train,
                         cv=cv,
                         scoring='accuracy',  # or 'f1_macro', etc.
                         n_jobs=1,
                         error_score='raise')
print("CV accuracy:", scores, "mean:", scores.mean())

Unused keyword parameter: n_jobs during cuML estimator initialization
Unused keyword parameter: n_jobs during cuML estimator initialization
Unused keyword parameter: n_jobs during cuML estimator initialization
Unused keyword parameter: n_jobs during cuML estimator initialization
Unused keyword parameter: n_jobs during cuML estimator initialization


MemoryError: std::bad_alloc: out_of_memory: CUDA error (failed to allocate 2265207984 bytes) at: /home/tyler/miniconda3/envs/kpop_project/include/rmm/mr/device/managed_memory_resource.hpp:66: cudaErrorMemoryAllocation out of memory

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_neighbors': [3,5,7,9],
    'weights': ['uniform'],
    'metric': ['euclidean']
}
gs = GridSearchCV(
    knn, param_grid,
    cv=cv,
    scoring='accuracy',
    verbose=1,
    n_jobs=1
)
gs.fit(X_train, y_train)
print("Best params:", gs.best_params_)
print("Best CV score:", gs.best_score_)
best_knn = gs.best_estimator_

Unused keyword parameter: n_jobs during cuML estimator initialization
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Unused keyword parameter: n_jobs during cuML estimator initialization
Unused keyword parameter: n_jobs during cuML estimator initialization
Unused keyword parameter: n_jobs during cuML estimator initialization
Unused keyword parameter: n_jobs during cuML estimator initialization
Unused keyword parameter: n_jobs during cuML estimator initialization
Unused keyword parameter: n_jobs during cuML estimator initialization
Unused keyword parameter: n_jobs during cuML estimator initialization
Unused keyword parameter: n_jobs during cuML estimator initialization
Unused keyword parameter: n_jobs during cuML estimator initialization
Unused keyword parameter: n_jobs during cuML estimator initialization
Unused keyword parameter: n_jobs during cuML estimator initialization
Unused keyword parameter: n_jobs during cuML estimator initialization
Unused keyword parameter: n_jo

/home/tyler/miniconda3/envs/kpop_project/lib/python3.10/site-packages/sklearn/model_selection/_validation.py:516: FitFailedWarning: 
15 fits failed out of a total of 20.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
12 fits failed with the following error:
Traceback (most recent call last):
  File "/home/tyler/miniconda3/envs/kpop_project/lib/python3.10/site-packages/sklearn/model_selection/_validation.py", line 859, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/home/tyler/miniconda3/envs/kpop_project/lib/python3.10/site-packages/cuml/internals/api_decorators.py", line 216, in wrapper
    ret = func(*args, **kwargs)
  File "/home/tyler/miniconda3/envs/kpop_project/lib/python3.10/site-packages/cuml/interna

Unused keyword parameter: n_jobs during cuML estimator initialization


MemoryError: std::bad_alloc: out_of_memory: CUDA error (failed to allocate 2831349872 bytes) at: /home/tyler/miniconda3/envs/kpop_project/include/rmm/mr/device/managed_memory_resource.hpp:66: cudaErrorMemoryAllocation out of memory

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

y_pred = best_knn.predict(X_test)
print(classification_report(y_test, y_pred))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred))

In [ ]:
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt

train_sizes, train_scores, valid_scores = learning_curve(
    best_knn, X_train, y_train,
    cv=cv, scoring='accuracy',
    train_sizes=np.linspace(0.1,1.0,5),
    n_jobs=-1
)
plt.plot(train_sizes, train_scores.mean(axis=1), label="train")
plt.plot(train_sizes, valid_scores.mean(axis=1), label="validation")
plt.xlabel("Training examples"); plt.ylabel("Accuracy")
plt.legend(); plt.show()

In [ ]:
from sklearn.model_selection import cross_val_score

nested_scores = cross_val_score(
    gs, X, y,
    cv=StratifiedKFold(5, shuffle=True, random_state=1),
    scoring='accuracy',
    n_jobs=-1
)
print("Nested CV accuracy:", nested_scores.mean())